# Env setup

In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import boto3
import pandas as pd
import numpy as np
import configparser
import psycopg2
from faker import Faker
import constants as cons

# Reading configuration file and credentials

In [20]:
config = configparser.ConfigParser()

config.read(cons.CONFIG_FILE)

['config.cfg']

In [11]:
aws_rds_conn = boto3.client('rds', aws_access_key_id=config.get(cons.USER, cons.ACCESS_KEY),
                            aws_secret_access_key=config.get(cons.USER, cons.SECRET_KEY),
                            region_name='us-east-1'
                            )

# Checking available instances for user

In [12]:
rds_instances_ids = []
aws_response = aws_rds_conn.describe_db_instances()

for response in aws_response['DBInstances']:
    rds_instances_ids.append(response['DBInstanceIdentifier'])

print(f'Available instances: {rds_instances_ids}')

Available instances: []


In [21]:
try: 
    response = aws_rds_conn.create_db_instance(
        DBInstanceIdentifier=config.get(cons.DB, cons.INSTANCE_ID),
        DBName=config.get(cons.DB, cons.DB_NAME),
        MasterUsername=config.get(cons.DB, cons.DB_USERNAME),
        MasterUserPassword=config.get(cons.DB, cons.DB_PASSWORD),
        Port=int(config.get(cons.DB, cons.PORT)),
        DBInstanceClass= cons.INSTANCE_CLASS,
        Engine=cons.ENGINE,
        PubliclyAccessible=True,
        AllocatedStorage=cons.ALLOC_STORAGE,
        VpcSecurityGroupIds=[config.get(cons.VPC, cons.SEC_GROUP)]
    )
except aws_rds_conn.exceptions.DBInstanceAlreadyExistsFault:
    print("La instancia ya existe")
except Exception as ex:
    print("Error!! ", ex)